In [5]:
# import required libraries

import pandas as pd
import joblib
import matplotlib.pyplot as plt
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")


#### Load Data and Model

In [3]:
# 📁 Load dataset
data_path = r"C:\Users\willi\GitHub\TS_Energy_Consumption_XGB\data\raw\PJME_hourly.csv"
PJME_data = pd.read_csv(data_path, index_col='Datetime', parse_dates=True)
PJME_data = PJME_data.sort_index()

# 🤖 Load trained XGBoost model
model_path = r"C:\Users\willi\GitHub\TS_Energy_Consumption_XGB\models\xgb_model.joblib"
reg_tuned = joblib.load(model_path)

# ✅ Confirm it worked
print("Data shape:", PJME_data.shape)
print("Model loaded:", type(reg_tuned))


Data shape: (145366, 1)
Model loaded: <class 'xgboost.sklearn.XGBRegressor'>


#### Forecast the Next 14 Days (Hourly)

In [6]:
# 🔧 Feature builder thats must match training.

def create_features(df):
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['dayofyear'] = df.index.dayofyear
    df['year'] = df.index.year
    return df

# 📅 Create next 14 days of hourly timestamps
future_dates = pd.date_range(
    start=PJME_data.index.max() + pd.Timedelta(hours=1),
    periods=24 * 14,  # 14 days × 24 hours
    freq='H'
)

# 🔄 Build features
future_df = pd.DataFrame(index=future_dates)
future_df = create_features(future_df)

# 📊 Predict
FEATURES = ['hour', 'dayofweek', 'quarter', 'month', 'dayofyear', 'year']
future_df['forecast'] = reg_tuned.predict(future_df[FEATURES])


#### Combine Forecast with Recent Actuals & Export to CSV

In [7]:
# 🧹 Ensure the index is sorted
PJME_data = PJME_data.sort_index()

# 🕒 Get last 7 days of actuals before forecast starts
forecast_start = future_df.index.min()
actual_window = PJME_data.loc[:forecast_start].last("7D")

# 📦 Prepare actuals
actual_df = actual_window[['PJME_MW']].copy()
actual_df['type'] = 'actual'
actual_df.rename(columns={'PJME_MW': 'MW'}, inplace=True)

# 📦 Prepare forecast
forecast_export = future_df[['forecast']].copy()
forecast_export['type'] = 'forecast'
forecast_export.rename(columns={'forecast': 'MW'}, inplace=True)

# 🔗 Combine actuals + forecast
combined_df = pd.concat([actual_df, forecast_export])
combined_df.index.name = 'Datetime'

# 💾 Export to CSV for dashboard
export_path = r"C:\Users\willi\GitHub\TS_Energy_Consumption_XGB\reports\actual_forecast_combined.csv"
combined_df.to_csv(export_path)

print(f"✅ Combined CSV saved to:\n{export_path}")


✅ Combined CSV saved to:
C:\Users\willi\GitHub\TS_Energy_Consumption_XGB\reports\actual_forecast_combined.csv
